In [16]:
root = "../../../gaia/runs/"
r1 = "../../../gaia/runs/gpt4o_mini_2/l{level}_gpt-4o-mini-2024-07-18_run.json"
r2 = "../../../gaia/runs/gpt4o_mini_t02/l{level}_gpt-4o-mini-2024-07-18_run.json"
r3 = "../../../gaia/runs/gpt4o_mini_t05/l{level}_gpt-4o-mini-2024-07-18_run.json"
# r3 = "../../../gaia/runs/gpt4o_mini_t02_2/l1_gpt-4o-mini-2024-07-18_run.json"


In [5]:
import sys

sys.path.append(".")


In [17]:
from eval import GaiaResults, majority_vote, tape_correct

from tapeagents.io import load_tapes

lvl1 = []
for fname in [r1, r2, r3]:
    lvl1.append(load_tapes(GaiaResults, fname.format(level="1"), file_extension=".json")[0])

lvl2 = []
for fname in [r1, r2, r3]:
    lvl2.append(load_tapes(GaiaResults, fname.format(level="2"), file_extension=".json")[0])

lvl3 = []
for fname in [r1, r2, r3]:
    lvl3.append(load_tapes(GaiaResults, fname.format(level="3"), file_extension=".json")[0])


In [18]:
top_tapes = []
avg = [[], [], [], []]
for l, lvl in enumerate([lvl1, lvl2, lvl3]):
    acc1 = []
    acc2 = []
    acc3 = []
    acc = []
    for i, tape in enumerate(lvl[0].tapes):
        result1 = tape["metadata"]["result"]
        result2 = lvl[1].tapes[i]["metadata"]["result"]
        result3 = lvl[2].tapes[i]["metadata"]["result"]
        tapes = [tape, lvl[1].tapes[i], lvl[2].tapes[i]]
        # print(result1, result2, result3)
        results = [result1, result2, result3]
        best_idx = majority_vote(results)
        best = results[best_idx]
        best_tape = tapes[best_idx]
        acc1.append(int(tape_correct(tape)))
        acc2.append(int(tape_correct(lvl[1].tapes[i])))
        acc3.append(int(tape_correct(lvl[2].tapes[i])))
        acc.append(int(tape_correct(best_tape)))
        top_tapes.append(tapes[best_idx])
        # print(best)
        # print()
    print(f"L{l+1} Accuracy 1: {sum(acc1) / len(acc1):.3f} ({sum(acc1)} of {len(acc1)})")
    print(f"L{l+1} Accuracy 2: {sum(acc2) / len(acc2):.3f} ({sum(acc2)} of {len(acc2)})")
    print(f"L{l+1} Accuracy 3: {sum(acc3) / len(acc3):.3f} ({sum(acc3)} of {len(acc3)})")
    print(f"L{l+1} Accuracy maj: {sum(acc) / len(acc):.3f} ({sum(acc)} of {len(acc)})")
    print()
    avg[0] += acc1
    avg[1] += acc2
    avg[2] += acc3
    avg[3] += acc

print(f"Avg. Accuracy 1: {sum(avg[0]) / len(avg[0]):.3f} ({sum(avg[0])} of {len(avg[0])})")
print(f"Avg. Accuracy 2: {sum(avg[1]) / len(avg[1]):.3f} ({sum(avg[1])} of {len(avg[1])})")
print(f"Avg. Accuracy 3: {sum(avg[2]) / len(avg[2]):.3f} ({sum(avg[2])} of {len(avg[2])})")
print(f"Avg. Accuracy maj: {sum(avg[3]) / len(avg[3]):.3f} ({sum(avg[3])} of {len(avg[3])})")


L1 Accuracy 1: 0.321 (17 of 53)
L1 Accuracy 2: 0.302 (16 of 53)
L1 Accuracy 3: 0.377 (20 of 53)
L1 Accuracy maj: 0.377 (20 of 53)

L2 Accuracy 1: 0.163 (14 of 86)
L2 Accuracy 2: 0.244 (21 of 86)
L2 Accuracy 3: 0.151 (13 of 86)
L2 Accuracy maj: 0.209 (18 of 86)

L3 Accuracy 1: 0.000 (0 of 26)
L3 Accuracy 2: 0.077 (2 of 26)
L3 Accuracy 3: 0.038 (1 of 26)
L3 Accuracy maj: 0.038 (1 of 26)

Avg. Accuracy 1: 0.188 (31 of 165)
Avg. Accuracy 2: 0.236 (39 of 165)
Avg. Accuracy 3: 0.206 (34 of 165)
Avg. Accuracy maj: 0.236 (39 of 165)


In [2]:
import json


In [3]:
with open("../../web_cache.json.old") as f:
    cache = json.load(f)
with open("../../web_cache2.json") as f:
    cache2 = json.load(f)
print(len(cache), len(cache2))


1152 879


In [4]:
full_cache = cache | cache2


In [5]:
files = []
pages = []
texts = []
for k, v in full_cache.items():
    if "http" in k:
        pages.append(k)
    elif "file://" in k:
        files.append(k)
    else:
        texts.append(k)
print(len(files), len(pages), len(texts))


38 348 1325


In [7]:
clean_cache = {k.lower(): v for k, v in full_cache.items() if len(v) > 0}
with open("../../web_cache.json", "w") as f:
    json.dump(clean_cache, f, indent=2, ensure_ascii=False)
